In [ ]:
import tsplib95
import numpy as np
import matplotlib.pyplot as plt
import math
import random

In [ ]:
def load_tsp_file_with_txt(file_path):
    problem = tsplib95.load(file_path)
    
    # get node coordinates
    coordinates = []
    for node in problem.get_nodes():
        coord = problem.node_coords[node]
        coordinates.append((coord[0], coord[1]))
    
    return np.array(coordinates)

# Load the data
cities = load_tsp_file_with_txt('eil51.txt')
print(np.shape(cities))
print("Loaded Cities' Coordinates:")
print(cities)


In [ ]:
def euclidean_distance(city1, city2):
    return np.linalg.norm(city1 - city2)

def compute_distance_matrix(cities):
    """Computes distance between two cities 
    Returns: Distance matrix 
    """
    num_cities = len(cities)
    dist_matrix = np.zeros((num_cities, num_cities))
    for i in range(num_cities):
        for j in range(num_cities):
            if i != j:
                dist_matrix[i][j] = euclidean_distance(cities[i], cities[j])
    return dist_matrix

distance_matrix = compute_distance_matrix(cities)

def calculate_total_distance(tour, dist_matrix):
    """Calculate total distance of one tour (defined as array of cities)"""
    total_distance = 0
    num_cities = len(tour)
    for i in range(num_cities):
        total_distance += dist_matrix[tour[i]][tour[(i + 1) % num_cities]]
    return total_distance

def generate_initial_solution(num_cities):
    """Initialize a random tour"""
    tour = list(range(num_cities))
    random.shuffle(tour)
    return tour


initial_tour = generate_initial_solution(len(cities))
print("Initial Tour:", initial_tour)
print("Initial Distance:", calculate_total_distance(initial_tour, distance_matrix))



In [ ]:
def two_opt_swap(tour, i, k):
    """Perform a 2-opt swap by reversing the segment between i and k"""
    new_tour = tour[:i] + tour[i:k + 1][::-1] + tour[k + 1:]
    return new_tour

def simulated_annealing(cities, initial_temperature, cooling_rate, max_iterations, markov_length):
    """Simulated annealing of travlling salesman
    
    Input: 
    cities (shape=(n,2) np.array) : coordinates of cities
    initial_temperature (float)
    cooling_rate: rate [0,1] of cooling per simulation step (slow cooling > 1, better result but slower)
    max_iterations: number of cooling steps
    markov_length: number of candidate solutions generated and evaluated at each temperature step
    
    Return:
    
    """
    num_cities = len(cities)
    dist_matrix = compute_distance_matrix(cities)
    # Generate random tour
    current_solution = generate_initial_solution(num_cities)
    current_distance = calculate_total_distance(current_solution, dist_matrix)
    best_solution = current_solution
    best_distance = current_distance
    
    temperature = initial_temperature

    iteration = 0
    #for iteration in range(max_iterations):
    while temperature > 0.1:
        iteration += 1
        for _ in range(markov_length):
            # Generate a new solution with a 2-opt move
            i, k = sorted(random.sample(range(num_cities), 2))
            new_solution = two_opt_swap(current_solution, i, k)
            new_distance = calculate_total_distance(new_solution, dist_matrix)
            
            # Calculate the change in distance
            delta = new_distance - current_distance

            # Accept the new solution based on the acceptance probability
            if delta < 0 or random.random() < math.exp(-delta / temperature):
                current_solution = new_solution
                current_distance = new_distance

                # Update the best solution if needed
                if current_distance < best_distance:
                    best_solution = current_solution
                    best_distance = current_distance

        # Cool down the temperature
        temperature *= cooling_rate

        # Optional: Print progress
        print(f"Iteration {iteration + 1}, Temperature: {temperature:.2f}, Best Distance: {best_distance:.2f}")

    return best_solution, best_distance


In [ ]:
best_tour, best_distance = simulated_annealing(
    cities=cities,
    initial_temperature=1000,
    cooling_rate=0.999,
    max_iterations=5000,
    markov_length=100
)

print("Best Tour:", best_tour)
print("Best Distance:", best_distance)


In [ ]:
def plot_tour(cities, tour):
    tour_cities = cities[tour + [tour[0]]]  # Complete the loop
    plt.figure(figsize=(10, 6))
    plt.plot(tour_cities[:, 0], tour_cities[:, 1], 'o-', markerfacecolor='red')
    plt.title("Best Tour Found")
    plt.xlabel("X Coordinate")
    plt.ylabel("Y Coordinate")
    plt.grid(True)
    plt.show()

plot_tour(cities, best_tour)


In [ ]:
def run_simulated_annealing_statistics(cities, initial_temperature, cooling_rate, max_iterations, markov_length, N):
    min_distances = []

    for i in range(N):
        _, best_distance = simulated_annealing(cities, initial_temperature, cooling_rate, max_iterations, markov_length)
        min_distances.append(best_distance)
        print(f"Run {i + 1}/{N}: Best Distance = {best_distance:.2f}")

    return min_distances

def plot_histogram(min_distances):
    plt.figure(figsize=(10, 6))
    plt.hist(min_distances, bins=15, color='skyblue', edgecolor='black')
    plt.title("Histogram of Minimum Distances Found Over 100 Runs")
    plt.xlabel("Distance")
    plt.ylabel("Frequency")
    plt.grid(True)
    plt.show()

initial_temperature = 2000
cooling_rate = 0.995
max_iterations = 1000
markov_length = 200
N = 25 # Number of runs

min_distances = run_simulated_annealing_statistics(cities, initial_temperature, cooling_rate, max_iterations, markov_length, N)

plot_histogram(min_distances)

In [ ]:
markov_lengths = [100, 200, 400]
cooling_rates = [0.99, 0.995, 0.999]
max_iterations_list = [1000, 2000]

num_runs = 25

for markov_length in markov_lengths:
    for cooling_rate in cooling_rates:
        for max_iterations in max_iterations_list:
            best_distances = []

            for _ in range(num_runs):
                _, best_distance = simulated_annealing(
                    cities=cities,
                    initial_temperature=2000,
                    cooling_rate=cooling_rate,
                    markov_length=markov_length,
                    max_iterations=max_iterations
                )
                best_distances.append(best_distance)

            # Plot histogram
            plt.figure(figsize=(10, 6))
            plt.hist(best_distances, bins=10, edgecolor='black', color='skyblue')
            min_distance = min(best_distances)
            plt.title(f'Histogram of Best Distances\nMarkov Length: {markov_length}, Cooling Rate: {cooling_rate}, Max Iterations: {max_iterations}')
            plt.xlabel('Best Distance')
            plt.ylabel('Frequency')
            plt.axvline(min_distance, color='red', linestyle='dashed', linewidth=1)
            plt.text(min_distance, plt.ylim()[1] * 0.9, f'Min Distance: {min_distance:.2f}', color='red', fontsize=12)
            plt.grid(True)
            plt.show()